## Descripción

El coronavirus tomó al mundo entero por sorpresa, cambiando la rutina diaria de todos y todas. Los habitantes de las ciudades ya no pasaban su tiempo libre fuera, yendo a cafés y centros comerciales; sino que más gente se quedaba en casa, leyendo libros. Eso atrajo la atención de las startups (empresas emergentes) que se apresuraron a desarrollar nuevas aplicaciones para los amantes de los libros.

## Objetivo 

Utilizar la información recopilada para generar una propuesta de valor para un nuevo producto.

In [1]:
# importar librerías
import pandas as pd
from sqlalchemy import create_engine, text


db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Mostrar cada una de las tablas 

In [2]:
query = """
SELECT *
FROM books
LIMIT 5
"""
results = pd.read_sql_query(sql=text(query), con=engine.connect())
display(results)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
query = """
SELECT *
FROM authors
LIMIT 5
"""
results = pd.read_sql_query(sql=text(query), con=engine.connect())
display(results)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
query = """
SELECT *
FROM ratings
LIMIT(5);
"""
results = pd.read_sql_query(sql=text(query), con=engine.connect())
display(results)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [5]:
query = """
SELECT *
FROM reviews;
"""
results = pd.read_sql_query(sql=text(query), con=engine.connect())
display(results)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


In [6]:
query = """
SELECT *
FROM publishers
LIMIT 5
"""
results = pd.read_sql_query(sql=text(query), con=engine.connect())
display(results)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Ejercicio

- Encuentra el número de libros publicados después del 1 de enero de 2000.

In [13]:
query_1 = """
SELECT COUNT(*) as count_books
FROM books
WHERE publication_date::date > '2000-01-01';
"""

results = pd.read_sql_query(sql=text(query_1), con=engine.connect())
display(results)

,count_books
0,819


- Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.

In [12]:
query_2 = """
SELECT 
    books.book_id,
    authors.author,
    books.title,
COUNT
    (distinct review_id) as number_users_reviews,
ROUND
    (COALESCE(AVG(ratings.rating), 0), 2) as average_rating
FROM books
    left join reviews on books.book_id = reviews.book_id
    left join ratings on books.book_id = ratings.book_id
    left join authors on books.author_id = authors.author_id
group by 
    books.book_id, authors.author_id
order by 
    average_rating desc
    
"""

results = pd.read_sql_query(sql=text(query_2), con=engine.connect())
display(results)

,book_id,author,title,number_users_reviews,average_rating
0,86,Mercedes Lackey,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,901,Robert Kirkman/Tony Moore/Charlie Adlard/Cliff...,The Walking Dead Book One (The Walking Dead #...,2,5.00
2,390,William Faulkner,Light in August,2,5.00
3,972,Jon Kabat-Zinn,Wherever You Go There You Are: Mindfulness Me...,2,5.00
4,136,John Eldredge/Stasi Eldredge,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
...,...,...,...,...,...
995,915,Thomas L. Friedman,The World Is Flat: A Brief History of the Twen...,3,2.25
996,316,Joseph J. Ellis,His Excellency: George Washington,2,2.00
997,202,Christina Schwarz,Drowning Ruth,3,2.00
998,371,William S. Burroughs/Oliver Harris/Allen Gins...,Junky,2,2.00


- Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).

In [16]:
query_3 = """
SELECT 
    publishers.publisher,
    COUNT(*) as num_books
FROM 
    books
    INNER JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE 
    num_pages >50
GROUP BY 
    publishers.publisher
ORDER BY 
    num_books DESC
LIMIT 1;
"""

results = pd.read_sql_query(sql=text(query_3), con=engine.connect())
display(results)

,publisher,num_books
0,Penguin Books,42


- Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.

In [10]:
query_4 = """
SELECT 
    authors.author_id,
    authors.author,
    AVG(ratings.rating) AS avg_rating
FROM 
    books
    INNER JOIN authors ON authors.author_id = books.author_id
    INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY 
    authors.author_id,
    authors.author
HAVING 
    COUNT(ratings.rating) >= 50
ORDER BY 
    avg_rating DESC
LIMIT 1;
"""
results = pd.read_sql_query(sql=text(query_4), con=engine.connect())
display(results)

,author_id,author,avg_rating
0,130,Diana Gabaldon,4.3


- Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

In [11]:
query_5 = """
SELECT 
    AVG(sub_review_cnt.review_cnt) AS avg_review_cnt
FROM (
  SELECT COUNT(text) AS review_cnt
  FROM 
    reviews
WHERE username IN (
    SELECT username
    FROM ratings
    GROUP BY 
      username
    HAVING 
      COUNT(ratings) > 50
  )
GROUP BY 
    username
) AS sub_review_cnt
"""

results = pd.read_sql_query(sql=text(query_5), con=engine.connect())
display(results)

,avg_review_cnt
0,24.333333


## Conclusiones Generales

- Entre el primero 01-01-2000 y el 31-03-02020 se han publicado 819 libros.
- La editorial Penguin books es la que mayor cantidad de libros ha publicado. 
- La Autora Diana Gabaldon es la mejor calificada entre los usuarios. 
- Para los usuarios que calificaron más de 50 libros, la cantidad de reseñas promedio fue de 24 calificaciones.